In [122]:
from pyBWMD import vectorize
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn import metrics
from sklearn.linear_model import LogisticRegression, SGDClassifier

from multiprocessing import Pool
import tqdm

In this example, our goal is to show that BWMD can be used on string/textual data as well. It works best when the text is longer, which is true for the 20-news groups dataset. While a TfidfVectorizer will work better, BWMD is still decent when only a few classes are present. 

In [124]:
import bz2
lines = []
sentences = []
words = []
with bz2.open('../../enwik8.bz2', 'rb') as f:
    for line in f:
        lines.append(line.decode('utf-8').strip())
        sentences.extend(line.decode('utf-8').strip().split('.'))
        words.extend(line.decode('utf-8').strip().split(' '))
print(f'Loaded {len(lines)} lines, {len(sentences)} sentences, and {len(words)} words.')

embeds = vectorize(sentences[:300])
embeds = embeds.toarray()
import scipy
import tqdm
from scipy.spatial import distance
# handle different length sentences


Loaded 1128024 lines, 1922572 sentences, and 13648953 words.


In [142]:
import math
import tqdm
from numba import jit

@jit(nopython=True)
def pairs(A):
    d = np.ones((A.shape[0], A.shape[0]))
    for i in range(A.shape[0]):
        for j in range(A.shape[0]):
            if i == j:
                continue
            d[i][j] = np.linalg.norm(A[i] - A[j])
    return d

# get closest
d = pairs(embeds)

In [141]:
print(d)

[[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]]


In [152]:
idx = np.argmin(d)
# get i,j from index
i = idx // d.shape[0]
j = idx % d.shape[0]
path = [i, j]
search_d = d.copy()
search_d[:,i] = 1.0
search_d[:,j] = 1.0

while True:
    tmp = search_d
    tmp[j][i] = 1.0
    tmp[i][j] = 1.0
    i = np.argmin(tmp[j])
    j = i
    path.append(j)
    search_d[:,j] = 1.0
    if len(path) >= len(embeds):
        break

In [156]:

# get bwt of a string (burrows-wheeler transform
def bwt(s):
    s += '\0'  # Add a terminal character
    table = sorted(s[i:] + s[:i] for i in range(len(s)))  # Table of rotations of s
    last_column = [row[-1:] for row in table]  # Last characters of each row
    return last_column

def rle(s):
    last = s[0]
    count = 1
    rle = []
    for c in s[1:]:
        if c == last:
            count += 1
        else:
            rle.append((last, count))
            last = c
            count = 1
    rle.append((last, count))
    return rle

for i in path:
    print(''.join(bwt(sentences[i])))

iiemkdwi a
iiemkdwi a
iiemkdwi a
3w 
3 
cs 
 
 
 
 
 
 
 
 
o c
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
'' 
 
 
 
 
 
 
n sm
sc a
smx atisr
smx atisr
bil 
ue d
buzp 
ssm ei
h nslgei
sogi tceei
>t<r eecc/<sstl>f-eiaarste
><u ee>nnommssaarrDeeuuc/<
><u ee>nnommssaarrDeeuuc/<
><8s0 ee>nnmmssaaArreemuu/<
><s ee>nnmmsshmtaasrreeeruuJe/<
><74s5 ee>CnnJmmssmaaarreeeuu/<
>e<s ee> nnnrmmgssneaarrAEdeeluu/<
><a ee>innemmpttdWkssiaaeei/<ii
>e=4<"a ye">ippnnaaeccpmmkdWk iaa</ssieee
>ea=5<"k ye">ipptnnaaeccpmmkdWkl iaaa</ssiee e
>ei=9<"k ye">aipptnnaaeccMmmkkdWl iaaa</ssee e
>e=8<"i ye">aippnnaaeccMmmkkdW iaa</sseee
>e=2"<-a ye">ippnnaaeccMmmkd aa</sseee
>e=1"<-l ye">ppinnaaeccpmmkc aaa</ssSeee
>e=1<"k ye">ppTnnaaccmmkl aaa</sseee
>er=3<"k ye">pptnnaaccsmmkl aaa</sseUee e
>ep=3<"1k ye">pptnnaaccHmmkl aeaa</lssee e
>e=2<"1p ye">ppnnaaccHmmk eaa</lsseee
>e=2<"r ye">ppnnaaccsmmk aa</sseUeee
>e=6<"e ye">ppmnnaacgcmmka Iaa</sseee
>ee=7<"k ye">ppmtnnaacgcmmkal aIaa</ssee e
>el=1<10"k ye">ppttnnaaccmmkl aaaa</Pssoe

In [ ]:
newsgroups_train = ['This is a test', 'This i  1wfwwevwaszw               za œ111r11111114222s another test', 'This is a third test']

In [78]:
import torch
import random
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# Define the Genome class
class Genome:
    def __init__(self, matrix):
        self.matrix = matrix
        self.mutation_log = []

    def mutate(self):
        i, j = random.randint(0, self.matrix.shape[0] - 1), random.randint(0, self.matrix.shape[1] - 1)
        mutation = random.choice([-1, 1])
        self.matrix[i, j] += mutation
        mutation_type = "Increment" if mutation > 0 else "Decrement"
        self.mutation_log.append(f"{mutation_type} at ({i}, {j})")

# Define the Genetic Algorithm class
class GeneticAlgorithm:
    def __init__(self, target_matrix, population_size=50):
        self.target = target_matrix
        self.population_size = population_size
        self.population = self.initialize_population()

    def initialize_population(self):
        population = []
        for _ in range(self.population_size):
            # choose a random 
            matrix = self.target[torch]
            population.append(Genome(matrix))
        return population

    def fitness(self, genome):
        return -torch.sum(torch.abs(genome.matrix - self.target)).item()

    def select_parents(self):
        parents = []
        for _ in range(self.population_size // 2):
            contenders = random.sample(self.population, 3)
            contenders.sort(key=lambda genome: self.fitness(genome), reverse=True)
            parents.append(contenders[0])
        return parents

    def crossover(self, parent1, parent2):
        child_matrix = parent1.matrix.clone()
        crossover_point = random.randint(0, child_matrix.numel() - 1)
        child_matrix.view(-1)[crossover_point:] = parent2.matrix.view(-1)[crossover_point:]
        child = Genome(child_matrix)
        child.mutation_log = parent1.mutation_log + parent2.mutation_log
        return child

    def mutate_population(self, mutation_rate=0.1):
        for genome in self.population:
            if random.random() < mutation_rate:
                genome.mutate()

    def run_evolution(self, generations, ax, update_interval=1):
        plt.ion()

        for generation in range(generations):
            new_population = []
            parents = self.select_parents()
            for i in range(0, len(parents), 2):
                for _ in range(2):
                    child = self.crossover(parents[i], parents[i+1])
                    new_population.append(child)
            self.population = new_population
            self.mutate_population()

            if generation % update_interval == 0:
                best_genome = max(self.population, key=lambda genome: self.fitness(genome))
                ax[0].clear()
                ax[1].clear()
                ax[0].imshow(self.target.numpy(), cmap='viridis')
                ax[0].set_title('Target Matrix')
                ax[0].axis('off')
                ax[1].imshow(best_genome.matrix.numpy(), cmap='viridis')
                ax[1].set_title(f'Generation {generation}')
                ax[1].axis('off')
                plt.pause(0.1)
        
        plt.ioff()

# Example usage
M, N = 4, 6
target_matrix = torch.tensor([[0, 2, 5, 0, 2, 4],
                              [2, 2, 2, 2, 2, 2],
                              [1, 0, 4, 3, 1, 2],
                              [1, 0, 1, 0, 1, 1]], dtype=torch.float)

fig, axs = plt.subplots(1, 2, figsize=(10, 5))
ga = GeneticAlgorithm(target_matrix, population_size=100)
ga.run_evolution(10, axs, update_interval=3)
